In [26]:
# --- PIL 임포트 (NameError 방지) ---
try:
    from PIL import Image, ExifTags
except Exception:
    import PIL.Image as Image
    ExifTags = None

GPS_TAG = 34853  # 0x8825

def has_gps_metadata(image_path: str) -> bool:
    try:
        im = Image.open(image_path)
        exif = im.getexif()
        return bool(exif) and (GPS_TAG in exif)
    except Exception:
        return False

def _ratio_to_float(x):
    # PIL EXIF (num, den) -> float
    try:
        return float(x[0]) / float(x[1])
    except Exception:
        return float(x)

def _dms_to_deg(values, ref) -> float:
    d = _ratio_to_float(values[0])
    m = _ratio_to_float(values[1])
    s = _ratio_to_float(values[2])
    deg = d + m/60.0 + s/3600.0
    if str(ref).upper() in ("S", "W"):
        deg = -deg
    return deg

def extract_gps(image_path: str):
    """
    반환: (lat, lon) 또는 (None, None)
    """
    try:
        im = Image.open(image_path)
        exif = im.getexif()
        gps = exif.get(GPS_TAG)
        if not gps:
            return (None, None)
        lat_ref = gps.get(1); lat_val = gps.get(2)
        lon_ref = gps.get(3); lon_val = gps.get(4)
        if not (lat_ref and lat_val and lon_ref and lon_val):
            return (None, None)
        lat = _dms_to_deg(lat_val, lat_ref)
        lon = _dms_to_deg(lon_val, lon_ref)
        return (lat, lon)
    except Exception:
        return (None, None)


In [27]:
pip install anthropic

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
import requests
import base64
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import anthropic
from io import BytesIO

 1. EXIF GPS 메타데이터 확인

In [29]:
def has_gps_metadata(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if not exif_data:
        return False
    for tag, value in exif_data.items():
        if TAGS.get(tag, tag) == "GPSInfo":
            return True
    return False

2. EXIF에서 위경도 추출

In [30]:
def get_exif_data(image):
    exif_data = {}
    info = image._getexif()
    if not info:
        return None
    for tag, value in info.items():
        tag_name = TAGS.get(tag, tag)
        if tag_name == "GPSInfo":
            gps_data = {}
            for t in value:
                sub_tag = GPSTAGS.get(t, t)
                gps_data[sub_tag] = value[t]
            exif_data['GPSInfo'] = gps_data
    return exif_data


def convert_to_degrees(value):
    d = value[0][0] / value[0][1]
    m = value[1][0] / value[1][1]
    s = value[2][0] / value[2][1]
    return d + (m / 60.0) + (s / 3600.0)


def extract_gps(image_path):
    image = Image.open(image_path)
    exif_data = get_exif_data(image)
    if not exif_data or 'GPSInfo' not in exif_data:
        return None, None
    gps_info = exif_data['GPSInfo']
    lat = convert_to_degrees(gps_info['GPSLatitude'])
    if gps_info['GPSLatitudeRef'] != 'N':
        lat = -lat
    lon = convert_to_degrees(gps_info['GPSLongitude'])
    if gps_info['GPSLongitudeRef'] != 'E':
        lon = -lon
    return lat, lon

3. 위경도로 장소명 추출 (Google Maps Geocoding)

In [31]:
def reverse_geocode(lat, lon, gmaps_api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lon}&key={gmaps_api_key}&language=ko"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data['status'] == 'OK':
            return data['results'][0]['formatted_address']
    return "장소 정보를 찾을 수 없습니다."

4. Claude 3 Opus로 장소 설명

In [32]:
def explain_place_with_claude(place_name, lat, lon, anthropic_api_key):
    prompt = f"""장소: {place_name}
위도: {lat}
경도: {lon}

이 장소에 대해 관광 정보, 역사, 위치 특징 등을 한국어로 자세히 설명해줘.
가능하다면 유명 인물이나 방송 촬영과 관련된 내용도 함께 말해줘."""

    client = anthropic.Anthropic(api_key=anthropic_api_key)
    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        temperature=0.5,
        system="당신은 위치 정보를 설명해주는 한국어 관광 안내 도우미입니다.",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.content[0].text.strip()

5. Claude 3 Opus로 이미지 직접 분석 (EXIF 없음)

In [33]:
def analyze_image_with_claude(image_path, anthropic_api_key):
    with open(image_path, "rb") as f:
        image_data = f.read()

    base64_image = base64.b64encode(image_data).decode("utf-8")
    client = anthropic.Anthropic(api_key=anthropic_api_key)

    with Image.open(image_path) as img:
     buffered = BytesIO()
     img.convert("RGB").save(buffered, format="JPEG")  # 강제 JPEG
     base64_image = base64.b64encode(buffered.getvalue()).decode()
    
    prompt_text = (
        "이 이미지를 보고 아래 항목들을 모두 가능한 범위 내에서 추정해줘:\n\n"
        "1. 사진 속 인물은 누구인지 (아이돌 이름과 소속 그룹)\n"
        "2. 어디에서 촬영되었는지 (장소명 + 위도/경도 포함 가능 시)\n"
        "3. 언제쯤 촬영된 사진인지 (연도, 계절 등)\n"
        "4. 어떤 프로그램/이벤트에서 찍힌 것인지\n"
        "5. 왜 촬영된 사진인지 (예: 방송, 팬미팅, 광고, 화보 등)\n\n"
        "정확하지 않더라도 추론 근거를 포함해서 한국어로 자연스럽게 설명해줘."
        "당신은 연예/아이돌 사진을 분석하는 전문가입니다. 사진 속 인물의 신원을 새로 특정하거나 이름을 추정하세요. "
        "배경 텍스트, 간판, 색채, 건물 형태, 로고/현수막, 의상/소품 등 비식별 단서를 근거로 장소/시간/이벤트/목적을 추정하세요. "
        "확신이 낮다면 표현에 신중함을 유지하세요.  "
        "이 이미지를 보고 아래 항목들을 모두 가능한 범위 내에서 추정해줘.\n\n"
        "1번은 무조건 적으로 어떤 아이돌인지 알려줘, 소속 그룹과 같이"
        "1. 사진 속 인물은 누구인지 (아이돌 이름과 소속 그룹)\n"
        "2. 어디에서 촬영되었는지, 장소를 최대한 정확하게 해줘 (장소명 + 위도/경도)\n"
        "3. 언제쯤 촬영된 사진인지 (연도, 계절 등)\n"
        "4. 어떤 프로그램/이벤트에서 찍힌 것인지\n"
        "5. 왜 촬영된 사진인지 (예: 방송, 팬미팅, 광고, 화보 등)\n\n"
        "정확하지 않더라도 추론 근거를 포함해서 한국어로 최대한 자연스럽게 설명해줘."
    )

    response = client.messages.create(
        model="claude-3-5-haiku-20241022",
        max_tokens=1024,
        temperature=0.6,
        system="당신은 사진을 보고 사람, 장소, 목적 등을 인식하는 시각 분석 전문가입니다.",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt_text},
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/jpeg",
                            "data": base64_image
                        }
                    }
                ]
            }
        ]
    )

    return response.content[0].text.strip()



드롭인 패치: 좌표/장소 추출 + 구글맵 링크 + 폴더 처리

In [34]:
import os, glob, re
from typing import Optional, Tuple
from urllib.parse import quote_plus

# ---------- 헬퍼: 숫자 첫 값 뽑기 ----------
def _first_number(x) -> Optional[float]:
    if isinstance(x, (int, float)): 
        return float(x)
    if isinstance(x, (list, tuple)):
        # [35.09], [35.09, 129.02], ["35.09"] 등 케이스
        for it in x:
            try:
                return float(it)
            except:
                continue
    try:
        return float(x)
    except:
        return None

def _norm_key(k: str) -> str:
    return k.lower().replace("_","").replace("-","")

# ---------- 좌표 뽑기(딕셔너리/리스트/문자열 모두 지원) ----------
def extract_lat_lng(obj) -> Tuple[Optional[float], Optional[float]]:
    lat = lng = None

    def try_parse_from_text(text: str):
        nonlocal lat, lng
        if lat is not None and lng is not None:
            return
        # 1) DMS (35° 05' 40.3" N, 129° 02' 15.0" E)
        m = re.search(
            r'(\d{1,2})\s*°\s*(\d{1,2})\s*[\'’]\s*(\d{1,2}(?:\.\d+)?)\s*"\s*([NS])\D+'
            r'(\d{1,3})\s*°\s*(\d{1,2})\s*[\'’]\s*(\d{1,2}(?:\.\d+)?)\s*"\s*([EW])',
            text, re.I
        )
        if m:
            def dms(d,mn,s,hemi):
                v = float(d) + float(mn)/60 + float(s)/3600
                return -v if hemi.upper() in ('S','W') else v
            lat = dms(m.group(1),m.group(2),m.group(3),m.group(4))
            lng = dms(m.group(5),m.group(6),m.group(7),m.group(8))
            return

        # 2) 소수 + N/E (35.0948° N, 129.0277° E)
        m = re.search(
            r'([-+]?\d{1,2}(?:\.\d+)?)\s*°?\s*([NS])\D+([-+]?\d{1,3}(?:\.\d+)?)\s*°?\s*([EW])',
            text, re.I
        )
        if m:
            a = float(m.group(1)); b = float(m.group(3))
            if m.group(2).upper() == 'S': a = -a
            if m.group(4).upper() == 'W': b = -b
            lat, lng = a, b
            return

        # 3) 한글 라벨 (위도: , 경도:)
        m = re.search(r'위도[:：]?\s*([-+]?\d{1,2}(?:\.\d+)?)\D+경도[:：]?\s*([-+]?\d{1,3}(?:\.\d+)?)', text)
        if m:
            a, b = float(m.group(1)), float(m.group(2))
            if -90 <= a <= 90 and -180 <= b <= 180:
                lat, lng = a, b
                return

        # 4) 일반 숫자쌍(최후 수단)
        m = re.search(r'([-+]?\d{1,2}(?:\.\d+)?)[,\s]+([-+]?\d{1,3}(?:\.\d+)?)', text)
        if m:
            a, b = float(m.group(1)), float(m.group(2))
            if -90 <= a <= 90 and -180 <= b <= 180:
                lat, lng = a, b

    def walk(o):
        nonlocal lat, lng
        if lat is not None and lng is not None:
            return
        if isinstance(o, dict):
            # 우선 lat/lng 키 직접 확인 (Claude가 숫자/리스트로 주는 케이스 포함)
            # 예: {"latitude": 35.09} 또는 {"latitude": [35.09], "longitude": [129.02]}
            for k, v in o.items():
                nk = _norm_key(k)
                if nk in ('lat','latitude') and lat is None:
                    lat = _first_number(v)
                elif nk in ('lng','lon','longitude') and lng is None:
                    lng = _first_number(v)

            # Google-style 중첩: geometry.location.lat / lng
            if lat is None or lng is None:
                loc = o.get('geometry',{}).get('location',{}) if isinstance(o.get('geometry'), dict) else {}
                if isinstance(loc, dict):
                    if lat is None: lat = _first_number(loc.get('lat'))
                    if lng is None: lng = _first_number(loc.get('lng'))

            # 좌표쌍 리스트: coordinates: [lng, lat] 혹은 [lat, lng]
            if lat is None or lng is None:
                coords = o.get('coordinates')
                if isinstance(coords, (list, tuple)) and len(coords) >= 2:
                    a = _first_number(coords[0])
                    b = _first_number(coords[1])
                    # 무엇이 lat인지 애매하면 범위로 추정
                    if a is not None and b is not None:
                        if -90 <= a <= 90 and -180 <= b <= 180:
                            lat, lng = a, b
                        elif -90 <= b <= 90 and -180 <= a <= 180:
                            lat, lng = b, a

            for v in o.values():
                walk(v)

        elif isinstance(o, (list, tuple)):
            for it in o:
                walk(it)

        elif isinstance(o, str):
            try_parse_from_text(o)

    walk(obj)
    return lat, lng

# ---------- 장소명으로 구글맵 검색 링크 (좌표 없을 때 폴백) ----------
def google_maps_search_from_place(obj) -> Optional[str]:
    candidates = []

    def walk(o):
        if isinstance(o, dict):
            for k, v in o.items():
                nk = _norm_key(k)
                if nk in ('placename','name','locationname','address','장소명','장소'):
                    if isinstance(v, str) and v.strip():
                        candidates.append(v.strip())
                walk(v)
        elif isinstance(o, (list, tuple)):
            for it in o: walk(it)
        elif isinstance(o, str):
            m = re.search(r'장소명[:：]\s*([^\n]+)', o)
            if m: candidates.append(m.group(1).strip())

    walk(obj)
    if candidates:
        return f"https://www.google.com/maps/search/?api=1&query={quote_plus(candidates[0])}"
    return None

def google_maps_link(lat: float, lng: float) -> str:
    return f"https://www.google.com/maps?q={lat:.6f},{lng:.6f}"


In [47]:
def run_with_maps(input_path: str, gmaps_api_key: str, anthropic_api_key: str):
    exts = (".jpg",".jpeg",".png",".webp",".bmp",".tif",".tiff",".gif")

    if os.path.isdir(input_path):
        paths = [os.path.join(input_path, f) for f in os.listdir(input_path)
                 if os.path.splitext(f)[1].lower() in exts]
        paths.sort()
    elif os.path.isfile(input_path):
        paths = [input_path]
    else:
        raise FileNotFoundError(f"입력 경로가 존재하지 않습니다: {input_path}")

    for p in paths:
        print(f"\n=== 파일: {p} ===")
        result = process_image(p, gmaps_api_key, anthropic_api_key)

        # ➊ 설명도 같이 출력
        if isinstance(result, dict):
            method = result.get("method")
            if method == "EXIF":
                print("📍 장소명:", result.get("place_name", "-"))
                print("📖 설명:", result.get("description", "(없음)"))
            else:  # Claude 등
                print("📖 분석:", result.get("place_description", "(없음)"))

        # ➋ 지도 링크 (좌표 우선 → 장소명 폴백)
        lat, lng = extract_lat_lng(result)
        if lat is not None and lng is not None:
            link = google_maps_link(lat, lng)
        else:
            link = google_maps_search_from_place(result)

        if link:
            print(f"🗺️ 구글 지도 링크: {link}")
        else:
            print("🗺️ 구글 지도 링크: 좌표/장소명을 찾지 못했습니다.")


6. 전체 분석 실행 함수

In [48]:
def process_image(image_path, gmaps_api_key, anthropic_api_key):
    print(f"\n📸 분석 중인 사진: {image_path}\n")

    if has_gps_metadata(image_path):
        lat, lon = extract_gps(image_path)
        if lat is None or lon is None:
            return {"method": "EXIF", "error": "GPS 추출 실패"}

        place_name = reverse_geocode(lat, lon, gmaps_api_key)
        description = explain_place_with_claude(place_name, lat, lon, anthropic_api_key)

        return {
            "method": "EXIF",
            "place_name": place_name,
            "latitude": lat,
            "longitude": lon,
            "description": description
        }
    else:
        result = analyze_image_with_claude(image_path, anthropic_api_key)
        return {
            "method": "Claude",
            "place_description": result
        }
    

사용 예시

In [ ]:
# image_path = "C:\\Users\\정하민\\Desktop\\덕픽 테스트\\stacy_busan.jpg"
# gmaps_api_key = "YOUR GOOGLE MAPS API KEY HERE"  # Google Maps API 키 (없으면 역지오코딩 생략)
# anthropic_api_key = "YOUR ANTHROPIC API KEY HERE"  # Anthropic API 키 (없으면 Claude 분석 생략)

# result = process_image(image_path, gmaps_api_key, anthropic_api_key)

# if result["method"] == "EXIF":
#     print("📍 장소명:", result["place_name"])
#     print("🧭 위도:", result["latitude"], "경도:", result["longitude"])
#     print("📖 설명:", result["description"])
# else:
#     print("📸 Claude 분석 결과:")
#     print(result["place_description"])

# ---------- 실행 예 ----------
if __name__ == "__main__":
    image_path_or_folder = r"C:\Users\정하민\Desktop\덕픽 테스트\data\seventeen_eiffel.jpg"  # 파일 또는 폴더
    gmaps_api_key = "YOUR GOOGLE MAPS API KEY HERE"  # Google Maps API 키 (없으면 역지오코딩 생략)
    anthropic_api_key = "YOUR ANTHROPIC API KEY HERE"  # Anthropic API 키 (없으면 Claude 분석 생략)

    # ❌ 이 줄은 폴더면 문제를 일으켜요: process_image(image_path_or_folder, ...)
    # ⭕ 폴더/파일을 모두 처리하려면 아래 한 줄만:
    run_with_maps(image_path_or_folder, gmaps_api_key, anthropic_api_key)



=== 파일: C:\Users\정하민\Desktop\덕픽 테스트\data\seventeen_eiffel.jpg ===

📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\seventeen_eiffel.jpg

📖 분석: 이미지 분석 결과를 상세히 설명드리겠습니다:

1. 사진 속 인물: 세븐틴(SEVENTEEN) 그룹 전체 멤버들

2. 촬영 장소: 프랑스 파리, 에펠탑 근처 유네스코(UNESCO) 본부 앞 광장
- 위도/경도: 약 48.8584° N, 2.2945° E

3. 촬영 시기: 2023년 여름 (맑은 하늘, 녹색 나무, 멤버들의 검은색 정장 스타일)

4. 이벤트: UNESCO 공식 행사 또는 기념식
- 배경의 유네스코 현수막, 다양한 국가 깃발 존재

5. 촬영 목적: 
- 공식 국제 행사 참석
- 문화 교류 또는 홍보 활동
- 국제기구와의 협력 관련 행사

특징:
- 멤버들 모두 통일된 검은색 정장 착용
- 많은 팬들이 스마트폰으로 사진 촬영 중
- 파리의 상징적 에펠탑이 배경에 있음
🗺️ 구글 지도 링크: https://www.google.com/maps?q=48.858400,2.294500


pdf 생성

In [51]:
# 주피터/콜랩에서는 %pip 가 현재 커널에 정확히 설치됩니다.
%pip install reportlab pillow

# 설치 확인
import reportlab, PIL
print("reportlab:", reportlab.Version, "| path:", reportlab.__file__)
print("Pillow:", PIL.__version__)


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.0 MB 330.3 kB/s eta 0:00:06
    --------------------------------------- 0.0/2.0 MB 653.6 kB/s eta 0:00:03
   ------ --------------------------------- 0.3/2.0 MB 3.2 MB/s eta 0:00:01
   ------------------ --------------------- 0.9/2.0 MB 7.2 MB/s eta 0:00:01
   -------------------------- ------------- 1.3/2.0 MB 7.4 MB/s eta 0:00:01
   --------------------------------- ------ 1.6/2.0 MB 8.1 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 8.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
reportlab: 4.4.3 | path: C:\Users\정하민\AppData\Roaming\Python\Python311\site-packages\reportlab\__init__.py
Pillow: 10.2.0


In [ ]:
import os, glob, re, json
from typing import List, Optional, Tuple
from urllib.parse import quote_plus

# ===== PIL / ReportLab =====
from PIL import Image, ImageOps
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4, landscape
from reportlab.lib.units import mm
from reportlab.lib.utils import ImageReader
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib import colors

# =======================
# 좌표/장소 → 구글맵 링크 헬퍼
# =======================
def _norm_key(k: str) -> str:
    return k.lower().replace("_","").replace("-","")

def _first_number(x) -> Optional[float]:
    if isinstance(x, (int, float)):
        return float(x)
    if isinstance(x, (list, tuple)):
        for it in x:
            try: return float(it)
            except: continue
    try: return float(x)
    except: return None

def extract_lat_lng(obj) -> Tuple[Optional[float], Optional[float]]:
    """딕트/리스트/문자열 안쪽까지 재귀 탐색해 위도·경도를 추출."""
    lat = lng = None

    def try_parse_from_text(text: str):
        nonlocal lat, lng
        if lat is not None and lng is not None:
            return
        # DMS 예: 35° 05' 40.3" N, 129° 02' 15.0" E
        m = re.search(
            r'(\d{1,2})\s*°\s*(\d{1,2})\s*[\'’]\s*(\d{1,2}(?:\.\d+)?)\s*"\s*([NS])\D+'
            r'(\d{1,3})\s*°\s*(\d{1,2})\s*[\'’]\s*(\d{1,2}(?:\.\d+)?)\s*"\s*([EW])', text, re.I)
        if m:
            def dms(d,mn,s,hemi):
                v = float(d) + float(mn)/60 + float(s)/3600
                return -v if hemi.upper() in ('S','W') else v
            lat = dms(m.group(1),m.group(2),m.group(3),m.group(4))
            lng = dms(m.group(5),m.group(6),m.group(7),m.group(8))
            return

        # 소수 + NSEW 예: 35.0948° N, 129.0277° E
        m = re.search(r'([-+]?\d{1,2}(?:\.\d+)?)\s*°?\s*([NS])\D+([-+]?\d{1,3}(?:\.\d+)?)\s*°?\s*([EW])', text, re.I)
        if m:
            a = float(m.group(1)); b = float(m.group(3))
            if m.group(2).upper() == 'S': a = -a
            if m.group(4).upper() == 'W': b = -b
            lat, lng = a, b
            return

        # 한글 라벨 예: 위도: 35.09, 경도: 129.02
        m = re.search(r'위도[:：]?\s*([-+]?\d{1,2}(?:\.\d+)?)\D+경도[:：]?\s*([-+]?\d{1,3}(?:\.\d+)?)', text)
        if m:
            a, b = float(m.group(1)), float(m.group(2))
            if -90 <= a <= 90 and -180 <= b <= 180:
                lat, lng = a, b
                return

        # 마지막 보루: 합리적 숫자쌍
        m = re.search(r'([-+]?\d{1,2}(?:\.\d+)?)[,\s]+([-+]?\d{1,3}(?:\.\d+)?)', text)
        if m:
            a, b = float(m.group(1)), float(m.group(2))
            if -90 <= a <= 90 and -180 <= b <= 180:
                lat, lng = a, b

    def walk(o):
        nonlocal lat, lng
        if lat is not None and lng is not None:
            return
        if isinstance(o, dict):
            # 1) 직계 키
            for k, v in o.items():
                nk = _norm_key(k)
                if nk in ('lat','latitude') and lat is None:
                    lat = _first_number(v)
                elif nk in ('lng','lon','longitude') and lng is None:
                    lng = _first_number(v)
            # 2) geometry.location 스타일
            if lat is None or lng is None:
                g = o.get('geometry')
                if isinstance(g, dict):
                    loc = g.get('location')
                    if isinstance(loc, dict):
                        if lat is None: lat = _first_number(loc.get('lat'))
                        if lng is None: lng = _first_number(loc.get('lng'))
            # 3) coordinates: [lat, lng] 또는 [lng, lat]
            if (lat is None or lng is None) and isinstance(o.get('coordinates'), (list, tuple)) and len(o['coordinates']) >= 2:
                a = _first_number(o['coordinates'][0])
                b = _first_number(o['coordinates'][1])
                if a is not None and b is not None:
                    if -90 <= a <= 90 and -180 <= b <= 180:
                        lat, lng = a, b
                    elif -90 <= b <= 90 and -180 <= a <= 180:
                        lat, lng = b, a
            # 4) 재귀
            for v in o.values():
                walk(v)
        elif isinstance(o, (list, tuple)):
            for it in o:
                walk(it)
        elif isinstance(o, str):
            try_parse_from_text(o)

    walk(obj)
    return lat, lng

def google_maps_link(lat: float, lng: float) -> str:
    return f"https://www.google.com/maps?q={lat:.6f},{lng:.6f}"

def google_maps_search_from_place(obj) -> Optional[str]:
    """좌표가 없을 때 장소명으로 검색 링크 생성."""
    cands = []
    def walk(o):
        if isinstance(o, dict):
            for k, v in o.items():
                nk = _norm_key(k)
                if nk in ('placename','name','locationname','address','장소명','장소'):
                    if isinstance(v, str) and v.strip(): cands.append(v.strip())
                walk(v)
        elif isinstance(o, (list, tuple)):
            for it in o: walk(it)
        elif isinstance(o, str):
            m = re.search(r'장소명[:：]\s*([^\n]+)', o)
            if m: cands.append(m.group(1).strip())
    walk(obj)
    if cands:
        return f"https://www.google.com/maps/search/?api=1&query={quote_plus(cands[0])}"
    return None

# =======================
# PDF 레이아웃 유틸
# =======================
def register_korean_font(font_path: Optional[str] = None) -> str:
    candidates = [
        font_path,
        r"C:\Windows\Fonts\malgun.ttf",
        r"C:\Windows\Fonts\NanumGothic.ttf",
        "/System/Library/Fonts/AppleSDGothicNeo.ttc",
        "/usr/share/fonts/truetype/nanum/NanumGothic.ttf",
        "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc",
    ]
    for p in candidates:
        if p and os.path.exists(p):
            try:
                pdfmetrics.registerFont(TTFont("KFont", p))
                return "KFont"
            except Exception:
                pass
    return "Helvetica"  # 최후 수단(한글 글리프 깨질 수 있음)

def wrap_text_by_width(cv: canvas.Canvas, text: str, max_width: float,
                       font_name: str, font_size: int) -> List[str]:
    lines = []
    for para in str(text).replace("\r", "").split("\n"):
        if not para:
            lines.append("")
            continue
        words = para.split(" ")
        cur = ""
        for w in words:
            trial = (cur + " " + w).strip()
            if cv.stringWidth(trial, font_name, font_size) <= max_width:
                cur = trial
            else:
                if cur: lines.append(cur)
                if cv.stringWidth(w, font_name, font_size) <= max_width:
                    cur = w
                else:
                    buf = ""
                    for ch in w:
                        if cv.stringWidth(buf+ch, font_name, font_size) <= max_width:
                            buf += ch
                        else:
                            lines.append(buf); buf = ch
                    cur = buf
        if cur: lines.append(cur)
    return lines

def draw_image_box(cv: canvas.Canvas, img_path: str, x: float, y: float,
                   box_w: float, box_h: float):
    im = Image.open(img_path)
    im = ImageOps.exif_transpose(im)  # EXIF 회전 보정
    iw, ih = im.size
    scale = min(box_w / iw, box_h / ih)
    dw, dh = iw * scale, ih * scale
    cx = x + (box_w - dw) / 2
    cy = y + (box_h - dh) / 2
    cv.drawImage(ImageReader(im), cx, cy, width=dw, height=dh, preserveAspectRatio=True, mask='auto')

def draw_page(cv: canvas.Canvas, img_path: str, description: str, index: int,
              font_name: str, map_url: Optional[str] = None, page_size=landscape(A4)):
    """한 페이지: [맨왼쪽 번호 | 왼쪽 사진 | 오른쪽 결과(상단에 파란 지도 링크)]"""
    width, height = page_size
    margin = 15*mm
    num_col_w = 14*mm
    gutter = 6*mm

    content_w = width - margin*2 - num_col_w - gutter
    image_col_w = content_w * 0.52
    text_col_w = content_w - image_col_w

    # 번호 뱃지
    circle_r = 6*mm
    circle_cx = margin + num_col_w/2
    circle_cy = height - margin - circle_r - 2*mm
    cv.setFillGray(0.92); cv.circle(circle_cx, circle_cy, circle_r, fill=1, stroke=0)
    cv.setFillGray(0.2); cv.setFont(font_name, 12)
    cv.drawCentredString(circle_cx, circle_cy - 4, str(index))

    # 이미지
    img_x = margin + num_col_w + gutter
    img_y = margin
    img_h = height - margin*2
    draw_image_box(cv, img_path, img_x, img_y, image_col_w, img_h)

    # 텍스트 준비
    text_x = img_x + image_col_w + gutter
    text_top = height - margin
    base_font_size = 12
    min_font_size = 9

    # 지도 링크 한 줄 공간
    link_block_h = 0
    link_font_size = 12
    link_text = "지도 열기 (클릭)"
    if map_url:
        link_block_h = link_font_size * 1.6

    # 줄바꿈 & 폰트 크기 조정
    avail_text_h = (height - margin*2) - link_block_h
    font_size = base_font_size
    while font_size >= min_font_size:
        lines = wrap_text_by_width(cv, description, text_col_w, font_name, font_size)
        leading = font_size * 1.4
        if leading * len(lines) <= avail_text_h:
            break
        font_size -= 1

    # 지도 링크(클릭 가능)
    y_cursor = text_top
    if map_url:
        cv.setFont(font_name, link_font_size)
        cv.setFillColor(colors.blue)
        w = cv.stringWidth(link_text, font_name, link_font_size)
        cv.drawString(text_x, y_cursor - link_font_size, link_text)
        pad = 2
        link_rect = (text_x - pad, y_cursor - link_font_size - pad,
                     text_x + w + pad, y_cursor - pad + 2)
        cv.linkURL(map_url, link_rect, relative=0)
        cv.line(text_x, y_cursor - link_font_size - 1, text_x + w, y_cursor - link_font_size - 1)
        cv.setFillGray(0.0)
        y_cursor -= link_block_h

    # 본문 텍스트
    cv.setFont(font_name, font_size)
    leading = font_size * 1.4
    y = y_cursor
    for line in lines:
        y -= leading
        if y < margin:
            cv.showPage()
            # 다음 페이지에도 "(계속)" 표시 + 동일 폰트
            cv.setFont(font_name, 10); cv.setFillGray(0.4)
            cv.drawString(margin, height - margin, f"{index}번 설명 (계속)")
            cv.setFillGray(0.0); cv.setFont(font_name, font_size)
            y = height - margin - leading
        cv.drawString(text_x, y, line)

# =======================
# 결과 텍스트 구성 (Claude/EXIF/Gemini 모두 커버)
# =======================
def compose_description_for_pdf(img_path: str, result: dict) -> Tuple[str, Optional[str]]:
    """
    오른쪽 '결과' 영역에 넣을 텍스트 + map_url(있으면)을 만들어 반환.
    """
    method = result.get("method") if isinstance(result, dict) else None

    # 좌표/링크 우선 생성
    lat, lng = extract_lat_lng(result)
    if lat is not None and lng is not None:
        map_url = google_maps_link(lat, lng)
    else:
        map_url = google_maps_search_from_place(result)

    # 본문 구성
    lines = []
    lines.append(f"📸 파일: {img_path}")
    if method == "EXIF":
        lines.append("🔎 분석 방법: EXIF")
        place = result.get("place_name") or result.get("address")
        if place: lines.append(f"📍 장소명: {place}")
        if lat is not None and lng is not None:
            lines.append(f"🧭 위도/경도: {lat:.6f}, {lng:.6f}")
        desc = result.get("description")
        if desc:
            lines.append("")
            lines.append("📖 설명:")
            lines.append(str(desc))
    elif method == "Claude":
        lines.append("🔎 분석 방법: Claude")
        desc = result.get("place_description") or result.get("description") or ""
        if lat is not None and lng is not None:
            lines.append(f"🧭 위도/경도: {lat:.6f}, {lng:.6f}")
        lines.append("")
        lines.append("📖 Claude 분석 결과:")
        lines.append(str(desc))
    else:
        # 알 수 없는 스키마: 가능한 키를 모아서 출력
        if lat is not None and lng is not None:
            lines.append(f"🧭 위도/경도: {lat:.6f}, {lng:.6f}")
        lines.append("")
        lines.append("📖 분석 결과:")
        lines.append(json.dumps(result, ensure_ascii=False, indent=2) if isinstance(result, dict) else str(result))

    return "\n".join(lines), map_url

# =======================
# 폴더/단일 → PDF 내보내기
# =======================
def export_images_to_pdf_with_results(input_path: str,
                                      gmaps_api_key: str,
                                      anthropic_api_key: str,
                                      output_pdf: str,
                                      font_path: Optional[str] = None):
    """
    맨왼쪽 = 번호, 왼쪽 = 사진, 오른쪽 = 결과 텍스트(+ 상단 파란 지도 링크)
    """
    # 이미지 목록 수집
    exts = (".jpg",".jpeg",".png",".webp",".bmp",".tif",".tiff",".gif")
    if os.path.isdir(input_path):
        image_paths = [os.path.join(input_path, f) for f in os.listdir(input_path)
                       if os.path.splitext(f)[1].lower() in exts]
        image_paths.sort()
    elif os.path.isfile(input_path):
        image_paths = [input_path]
    else:
        raise FileNotFoundError(f"입력 경로가 존재하지 않습니다: {input_path}")

    if not image_paths:
        raise ValueError("이미지 파일을 찾지 못했습니다.")

    # PDF 시작
    font_name = register_korean_font(font_path)
    cv = canvas.Canvas(output_pdf, pagesize=landscape(A4))

    for idx, img_path in enumerate(image_paths, start=1):
        # ★ 여기에 당신의 process_image 사용
        result = process_image(img_path, gmaps_api_key, anthropic_api_key)

        # 오른쪽 텍스트 & 지도 링크 생성
        description, map_url = compose_description_for_pdf(img_path, result)

        # 페이지 그리기
        draw_page(cv, img_path, description, idx, font_name, map_url=map_url)

        # 다음 이미지를 위해 페이지 넘김
        cv.showPage()

    cv.save()
    print(f"PDF 저장 완료: {output_pdf}")

# =======================
# 실행 예시
# =======================
if __name__ == "__main__":
    # (권장) 환경변수로 관리: GMAPS_API_KEY / ANTHROPIC_API_KEY
    gmaps_api_key = "YOUR GOOGLE MAPS API KEY HERE"  # Google Maps API 키 (없으면 역지오코딩 생략)
    anthropic_api_key = "   YOUR ANTHROPIC API KEY HERE"  # Anthropic API 키 (없으면 Claude 분석 생략)

    # 단일 파일 또는 폴더
    input_path = r"C:\Users\정하민\Desktop\덕픽 테스트\data"  # 폴더 예시
    # input_path = r"C:\Users\정하민\Desktop\덕픽 테스트\stacy_busan.jpg"  # 단일 파일

    output_pdf = r"C:\Users\정하민\Desktop\덕픽 테스트\결과.pdf"
    font_path = r"C:\Windows\Fonts\malgun.ttf"  # 한글 폰트 (필요 시 수정)

    export_images_to_pdf_with_results(input_path, gmaps_api_key, anthropic_api_key, output_pdf, font_path)



📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\aespa_bangkok.jpg


📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\bts_bomnal.jpg


📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\bts_buan.jpg


📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\ohmygirl_seunghui.jpg


📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\seventeen_eiffel.jpg


📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\seventeen_paris.jpg


📸 분석 중인 사진: C:\Users\정하민\Desktop\덕픽 테스트\data\stacy_busan.jpg

PDF 저장 완료: C:\Users\정하민\Desktop\덕픽 테스트\결과.pdf
